# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'10-02-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'10-02-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-10-03 04:23:40,33.93911,67.709953,39290,1458,32842,4990.0,Afghanistan,100.929038,3.710868
1,NaN,NaN,NaN,Albania,2020-10-03 04:23:40,41.15330,20.168300,13965,389,8342,5234.0,Albania,485.266523,2.785535
2,NaN,NaN,NaN,Algeria,2020-10-03 04:23:40,28.03390,1.659600,51847,1749,36385,13713.0,Algeria,118.234360,3.373387
3,NaN,NaN,NaN,Andorra,2020-10-03 04:23:40,42.50630,1.521800,2110,53,1540,517.0,Andorra,2730.861321,2.511848
4,NaN,NaN,NaN,Angola,2020-10-03 04:23:40,-11.20270,17.873900,5211,189,2215,2807.0,Angola,15.855162,3.626943


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,39145,39170,39186,39192,39227,39233,39254,39268,39285,39290
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,12787,12921,13045,13153,13259,13391,13518,13649,13806,13965
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,50400,50579,50754,50914,51067,51213,51368,51530,51690,51847


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1446,1451,1451,1453,1453,1455,1458,1458,1458,1458
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,370,370,373,375,377,380,384,387,388,389
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1698,1703,1707,1711,1714,1719,1726,1736,1741,1749


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,32610,32619,32619,32635,32642,32642,32746,32789,32842,32842
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,7139,7239,7309,7397,7397,7629,7732,7847,8077,8342
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,35428,35544,35654,35756,35860,35962,36063,36174,36282,36385


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1715,1738,1757,1764,1773,1785,1787,1791,1798,1805
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,5141,5165,5456,5477,5526,5588,5606,5640,5997,6024
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,851,857,873,882,885,886,886,896,898,902


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,25,25,25,25,25,25,27,27,28,27
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,49,50,50,50,50,50,50,52,53,53
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7,7,7,7,7,7,7,7,7,7


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
632,1001.0,Autauga,Alabama,US,2020-10-03 04:23:40,32.539527,-86.644082,1805,27,0,1778.0,"Autauga, Alabama, US",3230.771984,1.495845
669,1075.0,Lamar,Alabama,US,2020-10-03 04:23:40,33.779950,-88.096680,393,2,0,391.0,"Lamar, Alabama, US",2846.794640,0.508906
670,1077.0,Lauderdale,Alabama,US,2020-10-03 04:23:40,34.901719,-87.656247,1787,37,0,1750.0,"Lauderdale, Alabama, US",1927.120965,2.070509


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,7332200,208695,2873369
India,6394068,99773,5352078
Brazil,4847092,144680,4299659


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,7332200,208695,2873369,4250136,2020-10-03 04:23:40,37.923092,-91.345036
India,6394068,99773,5352078,942217,2020-10-03 04:23:40,23.088275,81.806127
Brazil,4847092,144680,4299659,402753,2020-10-03 04:23:40,-12.669522,-48.480493
Russia,1188928,20981,966724,201223,2020-10-03 04:23:40,54.546312,62.120860
Colombia,841532,26397,753953,61182,2020-10-03 04:23:40,5.349139,-74.190797


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,826013,16054,0
Texas,782306,16224,0
Florida,711804,14554,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,826013,16054,0,809959,2020-10-03 04:23:40,37.843962,-120.728594
Texas,782306,16224,0,766082,2020-10-03 04:23:40,31.660643,-98.653069
Florida,711804,14554,0,697250,2020-10-03 04:23:40,28.940755,-82.700744
New York,461629,33199,0,428430,2020-10-03 04:23:40,42.544151,-75.474183
Georgia,320634,7106,0,313528,2020-10-03 04:23:40,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,272653,6626,0
Florida,Miami-Dade,171391,3314,0
Illinois,Cook,146917,5239,0
Texas,Harris,146234,2615,0
Arizona,Maricopa,142566,3416,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,272653,6626,0,266027,2020-10-03 04:23:40,34.308284,-118.228241,6037.0
Florida,Miami-Dade,171391,3314,0,168077,2020-10-03 04:23:40,25.611236,-80.551706,12086.0
Illinois,Cook,146917,5239,0,141678,2020-10-03 04:23:40,41.841448,-87.816588,17031.0
Texas,Harris,146234,2615,0,143619,2020-10-03 04:23:40,29.858649,-95.393395,48201.0
Arizona,Maricopa,142566,3416,0,139150,2020-10-03 04:23:40,33.348359,-112.491815,4013.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-30,39268,13649,51530,2050,4972,101,751001,50359,27096,44813,...,455846,2046,56717,75122,1094,39899,10,2034,14759,7838
2020-10-01,39285,13806,51690,2050,5114,101,765002,50850,27109,45686,...,462775,2061,57190,76029,1095,40322,10,2039,14802,7850
2020-10-02,39290,13965,51847,2110,5211,106,779689,51382,27121,46374,...,469769,2097,57454,76820,1096,40766,10,2040,14830,7858


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-30,1458,387,1736,53,183,3,16937,959,888,799,...,42233,48,470,628,35,311,1,587,332,228
2020-10-01,1458,388,1741,53,185,3,20288,963,890,802,...,42292,48,471,635,35,318,1,587,333,228
2020-10-02,1458,389,1749,53,189,3,20599,966,893,803,...,42358,48,472,643,35,321,1,589,333,228


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-30,32789,7847,36174,1432,1941,92,594645,44001,24782,35644,...,2373,1791,53366,65225,1010,31743,8,1286,13959,6303
2020-10-01,32842,8077,36282,1432,2082,92,603140,44219,24786,36476,...,2383,1809,53835,66245,1010,32577,8,1297,13961,6312
2020-10-02,32842,8342,36385,1540,2215,94,614515,44406,24848,37186,...,2389,1824,53955,67216,1020,32944,8,1307,13980,6322


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1715,1738,1757,1764,1773,1785,1787,1791,1798,1805
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,357,363,366,367,370,377,382,382,386,393
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1673,1682,1698,1713,1723,1740,1751,1759,1772,1787
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,605,609,616,620,621,628,634,636,644,647
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,5792,5829,5887,5897,5927,5945,5971,6017,6043,6068


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-30,154701,7824,0,218507,83697,819115,70515,57550,20613,49,...,196139,773019,73042,1752,1318,148092,87522,15850,122274,5948
2020-10-01,155744,7948,0,219212,84821,822205,71197,57742,20787,49,...,197432,776736,74050,1755,1326,148536,88116,16026,125161,6083
2020-10-02,156698,8073,0,219763,85779,826013,71876,58297,20937,49,...,198403,782306,75157,1768,1326,149490,88810,16307,127906,6214


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-30        1791    5640     896  664   1629     610    917    3569   
2020-10-01        1798    5997     898  672   1634     612    919    3587   
2020-10-02        1805    6024     902  675   1642     612    922    3628   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-30         1179      620  ...     245     33      296      116   
2020-10-01         1181      624  ...     252     38      304      117   
2020-10-02         1185      631  ...     259     41      309      121   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-30            339   577   357          0      116     39  
2020-10-01            342   591   356          0      116     39  
2020-10-02            345   597   358          0      117     39  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-30,2540,56,0,5650,1369,15885,2051,4508,636,0,...,2454,16016,459,58,20,3205,2126,355,1327,50
2020-10-01,2548,57,0,5674,1384,15973,2054,4511,636,0,...,2501,16130,459,58,20,3225,2132,359,1348,53
2020-10-02,2550,57,0,5693,1391,16054,2057,4513,642,0,...,2515,16224,474,58,20,3247,2143,361,1353,53


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-30          27      52       7   11     15      15     40      44   
2020-10-01          28      53       7   10     15      16     40      44   
2020-10-02          27      53       7   10     15      15     40      44   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-30           42       13  ...       2      1        4        1   
2020-10-01           42       13  ...       2      1        4        1   
2020-10-02           42       13  ...       2      1        4        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-30              2     1     2          0        6      0  
2020-10-01              2     1     2          0        6      0  
2020-10-02              2     1     2          0        6      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-30,0.000357,0.009691,0.003154,0.042726,0.013660,0.000000,0.019538,0.009178,0.000665,0.017529,...,0.015860,0.006394,0.006441,0.010207,0.000000,0.009054,0.0,0.001477,0.002990,0.000128
2020-10-01,0.000433,0.011503,0.003105,0.000000,0.028560,0.000000,0.018643,0.009750,0.000480,0.019481,...,0.015200,0.007331,0.008340,0.012074,0.000914,0.010602,0.0,0.002458,0.002913,0.001531
2020-10-02,0.000127,0.011517,0.003037,0.029268,0.018968,0.049505,0.019199,0.010462,0.000443,0.015059,...,0.015113,0.017467,0.004616,0.010404,0.000913,0.011011,0.0,0.000490,0.001892,0.001019


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-30,0.0,0.007812,0.005794,0.0,0.022346,0.0,0.025304,0.001044,0.002257,0.003769,...,0.001684,0.0,0.008584,0.011272,0.0,0.016340,0.0,0.000000,0.000000,0.0
2020-10-01,0.0,0.002584,0.002880,0.0,0.010929,0.0,0.197851,0.004171,0.002252,0.003755,...,0.001397,0.0,0.002128,0.011146,0.0,0.022508,0.0,0.000000,0.003012,0.0
2020-10-02,0.0,0.002577,0.004595,0.0,0.021622,0.0,0.015329,0.003115,0.003371,0.001247,...,0.001561,0.0,0.002123,0.012598,0.0,0.009434,0.0,0.003407,0.000000,0.0


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-30,0.001313,0.014873,0.003078,0.132016,0.058920,0.000000,0.015000,0.003466,0.001293,0.020850,...,0.002111,0.011293,0.007476,0.015966,0.002979,0.022418,0.0,0.008627,0.001579,0.029566
2020-10-01,0.001616,0.029311,0.002986,0.000000,0.072643,0.000000,0.014286,0.004954,0.000161,0.023342,...,0.004214,0.010050,0.008788,0.015638,0.000000,0.026274,0.0,0.008554,0.000143,0.001428
2020-10-02,0.000000,0.032809,0.002839,0.075419,0.063881,0.021739,0.018860,0.004229,0.002501,0.019465,...,0.002518,0.008292,0.002229,0.014658,0.009901,0.011266,0.0,0.007710,0.001361,0.001584


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-30,0.007470,0.013209,NaN,0.001480,0.011383,0.004243,0.007300,0.003855,0.003994,0.0,...,0.007852,0.007501,0.012560,0.001715,0.00000,0.005070,0.005515,0.010004,0.019332,0.021818
2020-10-01,0.006742,0.015849,NaN,0.003226,0.013429,0.003772,0.009672,0.003336,0.008441,0.0,...,0.006592,0.004808,0.013800,0.001712,0.00607,0.002998,0.006787,0.011104,0.023611,0.022697
2020-10-02,0.006125,0.015727,NaN,0.002514,0.011294,0.004631,0.009537,0.009612,0.007216,0.0,...,0.004918,0.007171,0.014949,0.007407,0.00000,0.006423,0.007876,0.017534,0.021932,0.021535


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-09-30      0.002238  0.006065  0.011287  0.009119  0.004935  0.004942   
2020-10-01      0.003908  0.063298  0.002232  0.012048  0.003069  0.003279   
2020-10-02      0.003893  0.004502  0.004454  0.004464  0.004896  0.000000   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-09-30      0.000000  0.003656  0.003404  0.004862  ...  0.020833   
2020-10-01      0.002181  0.005043  0.001696  0.006452  ...  0.028571   
2020-10-02      0.003264  0.011430  0.003387  0.011218  ...  0.027778   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-09-30      0.031250  0.003390  0.074074   0.008929  0.030357  0.008475   
2020-10-01      0.151515  0.027027  0.008621   0.008850  0.024263 -0.002801   
2020-10-02      0.078947  0.016447  0.034188   0.008772  0.010152  0.005618   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-09-30            NaN  0.000000  0.054054  
2020-10-01            NaN  0.000000  0.000000  
2020-10-02            NaN  0.008621  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-30,0.009138,0.000000,NaN,0.003196,0.014074,0.006526,0.002444,0.000666,0.001575,NaN,...,0.014050,0.007169,0.004376,0.0,0.0,0.006279,0.000942,0.014286,0.020769,0.00
2020-10-01,0.003150,0.017857,NaN,0.004248,0.010957,0.005540,0.001463,0.000665,0.000000,NaN,...,0.019152,0.007118,0.000000,0.0,0.0,0.006240,0.002822,0.011268,0.015825,0.06
2020-10-02,0.000785,0.000000,NaN,0.003349,0.005058,0.005071,0.001461,0.000443,0.009434,NaN,...,0.005598,0.005828,0.032680,0.0,0.0,0.006822,0.005159,0.005571,0.003709,0.00


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                      \
Admin2           Autauga   Baldwin Barbour      Bibb Blount   Bullock Butler   
2020-09-30      0.000000  0.040000     0.0  0.100000    0.0  0.071429    0.0   
2020-10-01      0.037037  0.019231     0.0 -0.090909    0.0  0.066667    0.0   
2020-10-02     -0.035714  0.000000     0.0  0.000000    0.0 -0.062500    0.0   

Province_State                            ... Wyoming                  \
Admin2         Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2020-09-30         0.0      0.0      0.0  ...     0.0    0.0      0.0   
2020-10-01         0.0      0.0      0.0  ...     0.0    0.0      0.0   
2020-10-02         0.0      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-30          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-10-01          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-10-02          0.0        0.0   0.0   0.0        NaN      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-30,0.000412,0.009537,0.003089,0.031202,0.016552,0.002118,0.018396,0.007590,0.000653,0.015967,...,0.014896,0.007148,0.007990,0.010736,0.004592,0.009905,4.382561e-32,0.000819,0.003031,0.001050
2020-10-01,0.000422,0.010520,0.003097,0.015601,0.022556,0.001059,0.018520,0.008670,0.000566,0.017724,...,0.015048,0.007239,0.008165,0.011405,0.002753,0.010253,2.191280e-32,0.001639,0.002972,0.001291
2020-10-02,0.000275,0.011018,0.003067,0.022435,0.020762,0.025282,0.018859,0.009566,0.000504,0.016392,...,0.015081,0.012353,0.006391,0.010904,0.001833,0.010632,1.095640e-32,0.001065,0.002432,0.001155


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-30,0.000765,0.008226,0.004563,1.708035e-17,0.019493,1.346823e-51,0.024273,0.001890,0.003668,0.004464,...,0.001386,0.005405,0.009341,0.011733,1.481558e-10,0.019958,0.0,0.000028,0.000017,0.000578
2020-10-01,0.000383,0.005405,0.003721,8.540177e-18,0.015211,6.734114e-52,0.111062,0.003030,0.002960,0.004109,...,0.001391,0.002703,0.005735,0.011440,7.407792e-11,0.021233,0.0,0.000014,0.001515,0.000289
2020-10-02,0.000191,0.003991,0.004158,4.270089e-18,0.018416,3.367057e-52,0.063196,0.003073,0.003166,0.002678,...,0.001476,0.001351,0.003929,0.012019,3.703896e-11,0.015333,0.0,0.001711,0.000757,0.000144


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-30,0.001489,0.015451,0.002969,0.067044,0.045027,1.341437e-06,0.016075,0.003244,0.001890,0.020789,...,0.004313,0.010865,0.009298,0.015044,0.003783,0.027267,2.567907e-34,0.006640,0.003709,0.015891
2020-10-01,0.001553,0.022381,0.002978,0.033522,0.058835,6.707186e-07,0.015181,0.004099,0.001026,0.022065,...,0.004263,0.010458,0.009043,0.015341,0.001892,0.026770,1.283953e-34,0.007597,0.001926,0.008659
2020-10-02,0.000776,0.027595,0.002908,0.054470,0.061358,1.086990e-02,0.017020,0.004164,0.001764,0.020765,...,0.003391,0.009375,0.005636,0.014999,0.005896,0.019018,6.419766e-35,0.007653,0.001643,0.005122


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-30,0.006058,0.014594,NaN,0.001932,0.010148,0.003943,0.007408,0.004039,0.005154,8.473893e-61,...,0.006704,0.006834,0.012163,0.001980,0.000732,0.005169,0.005274,0.010836,0.019241,0.019408
2020-10-01,0.006400,0.015222,NaN,0.002579,0.011788,0.003858,0.008540,0.003688,0.006798,4.236947e-61,...,0.006648,0.005821,0.012982,0.001846,0.003401,0.004084,0.006031,0.010970,0.021426,0.021052
2020-10-02,0.006263,0.015474,NaN,0.002546,0.011541,0.004245,0.009038,0.006650,0.007007,2.118473e-61,...,0.005783,0.006496,0.013966,0.004627,0.001700,0.005253,0.006953,0.014252,0.021679,0.021294


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-09-30      0.003007  0.006873  0.006723  0.005741  0.003563  0.003227   
2020-10-01      0.003458  0.035085  0.004477  0.008894  0.003316  0.003253   
2020-10-02      0.003675  0.019794  0.004466  0.006679  0.004106  0.001627   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-09-30      0.000963  0.003713  0.003293  0.005264  ...  0.016552   
2020-10-01      0.001572  0.004378  0.002494  0.005858  ...  0.022562   
2020-10-02      0.002418  0.007904  0.002941  0.008538  ...  0.025170   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-09-30      0.044838  0.009642  0.063007   0.006430  0.019249  0.005714   
2020-10-01      0.098177  0.018335  0.035814   0.007640  0.021756  0.001457   
2020-10-02      0.088562  0.017391  0.035001   0.008206  0.015954  0.003537   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-09-30      -0.571428  0.002862  0.040224  
2020-10-01      -0.571428  0.001431  0.020112  
2020-10-02      -0.571428  0.005026  0.010056  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-30,0.006324,0.007022,NaN,0.002339,0.015044,0.006316,0.001812,0.000512,0.001535,NaN,...,0.011726,0.006153,0.005256,2.454060e-19,0.006579,0.005467,0.003522,0.015210,0.014252,0.000084
2020-10-01,0.004737,0.012440,NaN,0.003294,0.013000,0.005928,0.001637,0.000589,0.000767,NaN,...,0.015439,0.006635,0.002628,1.227030e-19,0.003290,0.005853,0.003172,0.013239,0.015039,0.030042
2020-10-02,0.002761,0.006220,NaN,0.003321,0.009029,0.005500,0.001549,0.000516,0.005101,NaN,...,0.010518,0.006231,0.017654,6.135151e-20,0.001645,0.006338,0.004166,0.009405,0.009374,0.015021


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2020-09-30      0.020083  0.020169  1.903665e-14  0.050029  0.000289   
2020-10-01      0.028560  0.019700  9.518326e-15 -0.020440  0.000144   
2020-10-02     -0.003577  0.009850  4.759163e-15 -0.010220  0.000072   

Province_State                                                        ...  \
Admin2           Bullock    Butler   Calhoun      Chambers  Cherokee  ...   
2020-09-30      0.035717  0.006514  0.002123  1.526071e-06 -0.002312  ...   
2020-10-01      0.051192  0.003257  0.001062  7.630357e-07 -0.001156  ...   
2020-10-02     -0.005654  0.001628  0.000531  3.815179e-07 -0.000578  ...   

Province_State   Wyoming                                                   \
Admin2              Park Platte  Sheridan Sublette Sweetwater Teton Uinta   
2020-09-30      0.000061    0.0  0.000092      0.0        0.0   0.0   0.0   
2020-10-01      0.000031    0.0  0.000046      0.0        0.0   0.0   0.0   
2020-10-02      0.000015    0.0  0.000023      0.0        0.0   0.0   0.0   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-09-30      -0.999969      0.0    NaN  
2020-10-01      -0.999969      0.0    NaN  
2020-10-02      -0.999969      0.0    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201002,AK,9040,460250,NaN,469290,41.0,NaN,NaN,NaN,...,469290,0,0,77ea689d232f21d7c3e86721a92703f37b818df6,0,0,0,0,0,NaN
1,20201002,AL,156698,1006612,NaN,1145601,752.0,17420.0,NaN,1840.0,...,1163310,2,163,38da52ec54bc8d9114fc1ef392985f137e8c9e1f,0,0,0,0,0,NaN
2,20201002,AR,85779,971951,NaN,1054260,473.0,5488.0,222.0,NaN,...,1057730,7,43,8d54985d8d3f135a0eca179098922a738522320d,0,0,0,0,0,NaN
3,20201002,AS,0,1616,NaN,1616,NaN,NaN,NaN,NaN,...,1616,0,0,674f4a4c04f6b26fbaa491a505588a65b45f829c,0,0,0,0,0,NaN
4,20201002,AZ,219763,1263417,NaN,1478504,586.0,22240.0,125.0,NaN,...,1483180,19,14,1ff269b7322edd0df769d60c47cfd1270df51253,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201002,AK,9040.0,460250.0,NaN,469290.0,41.0,NaN,NaN,NaN,...,469290,0,0,77ea689d232f21d7c3e86721a92703f37b818df6,0,0,0,0,0,NaN
1,20201002,AL,156698.0,1006612.0,NaN,1145601.0,752.0,17420.0,NaN,1840.0,...,1163310,2,163,38da52ec54bc8d9114fc1ef392985f137e8c9e1f,0,0,0,0,0,NaN
2,20201002,AR,85779.0,971951.0,NaN,1054260.0,473.0,5488.0,222.0,NaN,...,1057730,7,43,8d54985d8d3f135a0eca179098922a738522320d,0,0,0,0,0,NaN
3,20201002,AS,0.0,1616.0,NaN,1616.0,NaN,NaN,NaN,NaN,...,1616,0,0,674f4a4c04f6b26fbaa491a505588a65b45f829c,0,0,0,0,0,NaN
4,20201002,AZ,219763.0,1263417.0,NaN,1478504.0,586.0,22240.0,125.0,NaN,...,1483180,19,14,1ff269b7322edd0df769d60c47cfd1270df51253,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-10-02,AK,9040,460250,NaN,469290,41.0,NaN,NaN,NaN,6.0,...,469290,0,0,77ea689d232f21d7c3e86721a92703f37b818df6,0,0,0,0,0,NaN
2020-10-02,AL,156698,1006612,NaN,1145601,752.0,17420.0,NaN,1840.0,NaN,...,1163310,2,163,38da52ec54bc8d9114fc1ef392985f137e8c9e1f,0,0,0,0,0,NaN
2020-10-02,AR,85779,971951,NaN,1054260,473.0,5488.0,222.0,NaN,89.0,...,1057730,7,43,8d54985d8d3f135a0eca179098922a738522320d,0,0,0,0,0,NaN
2020-10-02,AS,0,1616,NaN,1616,NaN,NaN,NaN,NaN,NaN,...,1616,0,0,674f4a4c04f6b26fbaa491a505588a65b45f829c,0,0,0,0,0,NaN
2020-10-02,AZ,219763,1263417,NaN,1478504,586.0,22240.0,125.0,NaN,48.0,...,1483180,19,14,1ff269b7322edd0df769d60c47cfd1270df51253,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-10-02,AK,9040.0,460250.0,NaN,469290.0,41.0,NaN,NaN,NaN,6.0,...,469290,0,0,77ea689d232f21d7c3e86721a92703f37b818df6,0,0,0,0,0,NaN
2020-10-02,AL,156698.0,1006612.0,NaN,1145601.0,752.0,17420.0,NaN,1840.0,NaN,...,1163310,2,163,38da52ec54bc8d9114fc1ef392985f137e8c9e1f,0,0,0,0,0,NaN
2020-10-02,AR,85779.0,971951.0,NaN,1054260.0,473.0,5488.0,222.0,NaN,89.0,...,1057730,7,43,8d54985d8d3f135a0eca179098922a738522320d,0,0,0,0,0,NaN
2020-10-02,AS,0.0,1616.0,NaN,1616.0,NaN,NaN,NaN,NaN,NaN,...,1616,0,0,674f4a4c04f6b26fbaa491a505588a65b45f829c,0,0,0,0,0,NaN
2020-10-02,AZ,219763.0,1263417.0,NaN,1478504.0,586.0,22240.0,125.0,NaN,48.0,...,1483180,19,14,1ff269b7322edd0df769d60c47cfd1270df51253,0,0,0,0,0,NaN
